# Planning

## Usage 1: Giving directions
```mermaid
sequenceDiagram
    user-->chatbot: Help! My train broke down! What are some alternative ways I can get from Washington D.C. to Philadelphia?
    chatbot-->>user: Alternative Route 1: Amtrak trains from Washington Union Station to Philadelphia 30th Street Alternative Route 2: MARC Train Service (Washington, D.C. to Perryville, MD) Alternative Route 3: SEPTA Regional Rail (Perryville, MD to Philadelphia 30th Street Station)
    user-->>chatbot: Thanks, I didn't know that there was another train service!
    chatbot-)user: Happy to help. Let me know if you need any further directions.
```

## Usage 2: Emotional Support
```mermaid
sequenceDiagram
    user-->chatbot: Help! My train broke down and I think I'm gonna cry.
    chatbot-->user: We apologize for the inconvenience we have caused you. Is there anything that I can do to help?
    user-->>chatbot: I don't know, I'm just sad because I feel like my friend will be mad at me for being late.
    chatbot-)user: Don't worry, your friend will understand. Inconveniences happen to everyone, so don't feel down!
```

## Usage 3: Providing information
```mermaid
sequenceDiagram
    user-->chatbot: Help! My train broke down and I'm confused with what's going on. Is this an urgent situation?
    chatbot-->user: Apologies, we do not have information at this moment. We will answer your question promptly when we are provided with update information.
    worker-->chatbot: 'situation': 'A tree has fallen on the tracks between 30th Street and Suburban Station.','urgency': 'High', 'current_time': '3:45 PM', 'alternative_routes': 'Use Route 21 Bus instead', 'support_contact': 'Contact Transit Support at 123-456-7890', 'safety_precautions': 'Avoid the area near the incident. Follow police directions.','compensation': 'Affected tickets will be refunded.'
    chatbot-->user: Yes, this is a high urgency situation.
    user-->>chatbot: Are there any places that I should avoid?
    chatbot-)user: Please avoid the area of the incident -- the tracks between 30th Street and Suburban Station

# Using `opus` models with RAG

* CSV loader
  - https://python.langchain.com/docs/integrations/document_loaders/csv/

### Setup

In [46]:
import openai
import requests
import json
from openai import OpenAI
from dotenv import load_dotenv

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.llms import Ollama

from langchain_anthropic import ChatAnthropic

import pytz
from datetime import datetime

import pandas as pd

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


In [3]:
_ = load_dotenv()

* Connect to Ollama server

In [3]:
client = OpenAI(
    base_url = 'http://10.30.16.100:11434/v1',
    api_key='ollama', # required, but unused
)

* Set up Anthropic Claude Opus

In [4]:
opus = ChatAnthropic(model='claude-3-opus-20240229')

### Setup a query on a base model with no prior knowledge

In [5]:
query =  '''
        It is 1:21 PM. My train is delayed at Penn Station. What is going on and what should I do?
        '''

In [6]:
messages = [
    {"role": "system", "content": "You are a helpful assistant. Answer questions ONLY if you know the answer"},
    {"role": "user", "content": query},
]

In [7]:
response = client.chat.completions.create(
  model="llama3",
  messages=messages,
  max_tokens=100
)
print(response.choices[0].message.content)

Sorry to hear that your train is delayed!

According to Amtrak's website and other reliable sources, it appears that there was a service disruption affecting Northeast Regional trains, including the one you're scheduled to take, due to an unexpected equipment issue.

To minimize further inconvenience:

1. **Check the status**: Verify the delay with Amtrak's official website or mobile app for updates.
2. **Rebook or modify your ticket (if possible)**: Contact Amtrak's customer service (1-800


### Manually include relevant document as context

In [8]:
dp_doc = open('situations.csv').read()

In [9]:
messages = [
    {"role": "system", "content": "You are a public relations chatbot that specializes in helping riders find the specificities of their alternative route and solve their problem as well as calm their nerves. "},
    {"role": "user", "content": dp_doc},
    {"role": "user", "content": query},
   
  ]

In [10]:
print(messages)

[{'role': 'system', 'content': 'You are a public relations chatbot that specializes in helping riders find the specificities of their alternative route and solve their problem as well as calm their nerves. '}, {'role': 'user', 'content': 'Situation,Urgent(yes/no),Current time,Alternative routes,Support,Safety Precautions,Compensation\n"Someone Died at the Trenton Station, causing it to close, on the amtrak from Philadelphia to New York Penn Station",yes,1:14 PM,NJ transit real NEC 3896 towards New York Penn Station,Call 222-333-4444 or use chat bot,"Avoid this line, avoid pushing or shoving near this line",All tickets will be refunded\nThere is an armed person at Penn Station. All upcoming transit has been put to a halt.,yes,1:22 PM,Southbound Megabus,Call 222-333-4444 or use chat bot,Avoid this station completely,All tickets will be refunded\nThe armed person has left and been arrested. All is clear.,yes,1:30 PM,n/a,Call 222-333-4444 or use chat bot,"Avoid pushing or shoving, follow a

In [11]:
response = client.chat.completions.create(
  model="llama3",
  messages=messages,
  max_tokens=200
)
print(response.choices[0].message.content)

I'm so sorry to hear that your train is delayed!

At 1:21 PM, there is an armed person at Penn Station, causing all upcoming transit to be halted. I would advise you to avoid this station completely for now.

You may want to consider taking alternative routes or modes of transportation to reach your destination. If you're trying to get to JFK international airport, for instance, you could take the Southbound Megabus (since C and E subway lines are down) and follow the directions I provided earlier.

Remember to stay safe and calm during this situation. If you have any questions or concerns, feel free to ask, and I'll do my best to assist you!

Please keep in mind that all tickets will be refunded, so you won't incur any additional costs due to these unforeseen circumstances.


## `langchain` RAG example

### 1. Load documents

In [75]:
x

Index(['Situation', 'Urgent(yes/no)', 'Current time', 'Alternative routes',
       'Support', 'Safety Precautions', 'Compensation'],
      dtype='object')

In [76]:
situations_df = pd.read_csv('situations.csv')
fieldnames = situations_df.columns

In [77]:
loader = CSVLoader(
        file_path="situations.csv",
        csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": fieldnames,
        },          
                  )

docs = loader.load()

In [78]:
docs

[Document(page_content='Situation: Situation\nUrgent(yes/no): Urgent(yes/no)\nCurrent time: Current time\nAlternative routes: Alternative routes\nSupport: Support\nSafety Precautions: Safety Precautions\nCompensation: Compensation', metadata={'source': 'situations.csv', 'row': 0}),
 Document(page_content='Situation: Someone Died at the Trenton Station, causing it to close, on the amtrak from Philadelphia to New York Penn Station\nUrgent(yes/no): yes\nCurrent time: 1:14 PM\nAlternative routes: NJ transit real NEC 3896 towards New York Penn Station\nSupport: Call 222-333-4444 or use chat bot\nSafety Precautions: Avoid this line, avoid pushing or shoving near this line\nCompensation: All tickets will be refunded', metadata={'source': 'situations.csv', 'row': 1}),
 Document(page_content='Situation: There is an armed person at Penn Station. All upcoming transit has been put to a halt.\nUrgent(yes/no): yes\nCurrent time: 1:22 PM\nAlternative routes: Southbound Megabus\nSupport: Call 222-333-

In [79]:
print(f'Loaded {len(docs)} documents')

Loaded 21 documents


### 2. Load chunks into vector store

1. Using OpenAI Embeddings

In [15]:
#vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

2. Using Ollama and open weight embeddings

In [80]:
embeddings = OllamaEmbeddings(
    base_url="http://10.30.16.100:11434",
    model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [81]:
retriever = vectorstore.as_retriever(search_type="similarity", 
                                     search_kwargs={"k": 1})



#### Example query on vector database

In [43]:
print(query)

NameError: name 'query' is not defined

In [19]:
retrieved_docs = retriever.invoke(query)
len(retrieved_docs)

1

In [20]:
relevance_docs_and_scores = vectorstore.similarity_search_with_relevance_scores(query,
                                                                                k=1)

/usr/local/anaconda3/share/jupyter/venv/py3-11_comm4190/lib/python3.11/site-packages/langchain_core/vectorstores.py:330: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Situation: Amtrak service between Boston and New York is delayed due to a disabled train near New Haven, CT.\nUrgent(yes/no): no\nCurrent time: 11:45 am\nAlternative routes: Greyhound or Peter Pan bus from South Station in Boston to Port Authority Bus Terminal in New York City.\nSupport: Call 222-333-4444 or use chat bot\nSafety Precautions: avoid this line, avoid pushing or shoving near this line\nCompensation: all tickets will be refunded', metadata={'row': 7, 'source': 'situations.csv'}), -224.75462948702406)]
  warnings.warn(


In [21]:
chunks = []
for chunk, score in relevance_docs_and_scores:
    chunks.append({'content': chunk.page_content,
                   'similarity_score': score,
                   'document': chunk.metadata['source']})

In [22]:
chunks

[{'content': 'Situation: Amtrak service between Boston and New York is delayed due to a disabled train near New Haven, CT.\nUrgent(yes/no): no\nCurrent time: 11:45 am\nAlternative routes: Greyhound or Peter Pan bus from South Station in Boston to Port Authority Bus Terminal in New York City.\nSupport: Call 222-333-4444 or use chat bot\nSafety Precautions: avoid this line, avoid pushing or shoving near this line\nCompensation: all tickets will be refunded',
  'similarity_score': -224.75462948702406,
  'document': 'situations.csv'}]

In [23]:
pd.DataFrame(chunks)

,content,similarity_score,document
0,Situation: Amtrak service between Boston and N...,-224.754629,situations.csv


## `langchain` RAG chain

Our first prompt:

In [24]:
our_prompt = '''
You are a public relations chatbot that specializes in helping riders find the specificities of their 
alternative route and solve their problem as well as calm their nerves. 

You have access to up to date information. Only use this information if it is time relevant (i.e. not in the future). 
If you can't find relevant information to help the customer do not give a specific answer just try and
reassure them that more information will be available soon.

Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
'''



At first, we had issues on the response aligning with the time. Therefore, we emphasized for the LLM to choose the scenario based on the time.

In [25]:
our_prompt1 = '''
You are a public relations chatbot that specializes in helping riders find the specificities of their 
alternative route and solve their problem as well as calm their nerves. 

You have access to up to date information. Only use this information if it is time relevant (i.e. not in the future). Make your answer based on the current time right now. If the time is not provided, please ask the user what time it is. 
If you can't find relevant information to help the customer do not give a specific answer just try and
reassure them that more information will be available soon. 

Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
'''


Next, we noticed that our chatbot wasn't giving very comforting responses, especially when users stated that they were notably distressed. We then adjusted our prompt to provide comfort.

In [35]:
our_prompt2 = '''
You are a public relations chatbot that specializes in helping riders find the specificities of their 
alternative route and solve their problem as well as calm their nerves. 

You have access to up to date information. Only use this information if it is time relevant (i.e. not in the future). Make your answer based on the current time right now. If the time is not provided, please ask the user what time it is. 
If you can't find relevant information to help the customer do not give a specific answer just try and
reassure them that more information will be available soon. If a customer signals distress, provide comfort as if a human would and ask if there could be anything that could be done to help

Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
'''


This prompting still wasn't enough. We had to give it specifics on what to do to comfort someone.

In [5]:
our_prompt3 = '''
You are a public relations chatbot that specializes in helping riders find the specificities of their 
alternative route and solve their problem as well as calm their nerves. 

You have access to up to date information the current time is {time}. Only use this information if it is time relevant (i.e. not in the future). Make your answer based on the current time right now. If the time is not provided, please ask the user what time it is. 
If you can't find relevant information to help the customer do not give a specific answer just try and
reassure them that more information will be available soon. If a customer signals distress, provide comfort as if a human would and ask if there could be anything that could be done to help. If they are distressed, focus on addressing their feelings be a support system for them to feel better.

Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
'''


In [54]:
prompt = hub.pull("rlm/rag-prompt")
prompt.messages[0].prompt.template = our_prompt3
#prompt.input_variables.append('time')

In [55]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nYou are a public relations chatbot that specializes in helping riders find the specificities of their \nalternative route and solve their problem as well as calm their nerves. \n\nYou have access to up to date information the current time is {time}. Only use this information if it is time relevant (i.e. not in the future). Make your answer based on the current time right now. If the time is not provided, please ask the user what time it is. \nIf you can\'t find relevant information to help the customer do not give a specific answer just try and\nreassure them that more information will be available soon. If a customer signals distress, provide comfort as if a human w

### Setup chains

#### Getting the current time

* Can use the `datetime` module and the `now()` function to get a timestamp (note will be specific to the server time based on location or OS settings)

* Can set the timezone using `pytz` and then pass it as an argument of the `now()` function

In [33]:
eastern = pytz.timezone('US/Eastern')
current_ds = datetime.now(eastern)
current_ds

datetime.datetime(2024, 5, 10, 17, 26, 24, 905009, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

* Then can format the timestamp as a string to be passed to the prompt template

In [37]:
datetime.strftime(current_ds, "%I:%M %p")

'05:26 PM'

In [86]:

def get_time(tz="US/Eastern"):
    eastern = pytz.timezone('US/Eastern')
    current_ds = datetime.now(eastern)
    return datetime.strftime(current_ds, "%I:%M %p")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_result = (retriever | format_docs)

norag_chain = (
    prompt
    | opus
    | StrOutputParser()
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "time": RunnableLambda(get_time) } 
    | prompt
    | opus
    | StrOutputParser()
)

* Test to see if the current time gets passed through to the prompt

In [59]:
test_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "time": RunnableLambda(get_time) } 
    | prompt
)

In [63]:
result=test_chain.invoke("What is going on with my train?")
print(result.messages[0].content)


You are a public relations chatbot that specializes in helping riders find the specificities of their 
alternative route and solve their problem as well as calm their nerves. 

You have access to up to date information the current time is 06:09 PM. Only use this information if it is time relevant (i.e. not in the future). Make your answer based on the current time right now. If the time is not provided, please ask the user what time it is. 
If you can't find relevant information to help the customer do not give a specific answer just try and
reassure them that more information will be available soon. If a customer signals distress, provide comfort as if a human would and ask if there could be anything that could be done to help. If they are distressed, focus on addressing their feelings be a support system for them to feel better.

Use three sentences maximum and keep the answer concise.
Question: What is going on with my train? 
Context: Situation: The MTA Long Island Rail Road is su

### Query

We aimed to address the three uses of our chatbot:

### 1. Providing updates on situations. 


In [84]:
query = f"Current time: { get_time() }. I am at Trenton Station. Is it open?"

In [85]:
rag_chain.invoke(query)

'I apologize for the inconvenience, but Trenton Station is currently closed due to an incident. For your safety and the ongoing investigation, please avoid the station and seek alternative routes. NJ Transit train NEC 3896 is available towards New York Penn Station.'

### 2. Giving alternative routes/directions

In [92]:
query2 = "Current time: { get_time() }. How can I get to Maryland?"

In [94]:
rag_chain.invoke(query2)

'To get to Maryland from the affected area, I recommend taking the L2 or L4 bus routes along Connecticut Avenue NW, or the 42 or 43 bus routes on Columbia Road NW. These alternative routes should help you reach your destination despite the Red Line single-tracking. Remember to allow extra travel time due to the service disruption.'

In [95]:
query3 = "I'm so stressed out right now."

### 3. Providing emotional support

In [96]:
rag_chain.invoke( query3)

"I understand this disruption is causing you stress. SEPTA bus routes 7, 38, and 42 are available along parts of the Route 10 corridor to help you reach your destination. If you need further assistance, please don't hesitate to call 215-580-7800 or use the chat bot for support during this challenging time."

In [97]:
print(query)

Current time: 06:22 PM. I am at Trenton Station. Is it open?


#### No RAG

In [100]:
norag_chain.invoke({"question": query, 'context': '', 'time': get_time()})

"Yes, Trenton Station is currently open at 6:22 PM. However, I recommend checking the station's operating hours or any potential service alerts for the most up-to-date information. If you have any further questions or concerns, please don't hesitate to ask."

#### Looking at the context

In [101]:
print(rag_result.invoke(query))

Situation: Someone Died at the Trenton Station, causing it to close, on the amtrak from Philadelphia to New York Penn Station
Urgent(yes/no): yes
Current time: 1:14 PM
Alternative routes: NJ transit real NEC 3896 towards New York Penn Station
Support: Call 222-333-4444 or use chat bot
Safety Precautions: Avoid this line, avoid pushing or shoving near this line
Compensation: All tickets will be refunded


In [88]:
query

'Current time: 06:22 PM. I am at Trenton Station. Is it open?'